In [31]:
import polars as pl

In [94]:
import re


def find_ngrams(text: str, number: int=3) -> set:
    """
    returns a set of ngrams for the given string
    :param text: the string to find ngrams for
    :param number: the length the ngrams should be. defaults to 3 (trigrams)
    :return: set of ngram strings
    """

    if not text:
        return set()

    words = [f'  {x} ' for x in re.split(r'\W+', text.lower()) if x.strip()]

    ngrams = set()

    for word in words:
        for x in range(0, len(word) - number + 1):
            ngrams.add(word[x:x+number])

    return ngrams


def similarity(text1: str, text2: str, number: int=3) -> float:
    """
    Finds the similarity between 2 strings using ngrams.
    0 being completely different strings, and 1 being equal strings
    """

    ngrams1 = find_ngrams(text1, number)
    ngrams2 = find_ngrams(text2, number)

    num_unique = len(ngrams1 | ngrams2)
    num_equal = len(ngrams1 & ngrams2)

    return float(num_equal) / float(num_unique)


def only_similarity(text1: str, text2: str, number: int=3) -> float:
    """
    Finds the similarity between 2 strings using ngrams.
    0 being completely different strings, and 1 being equal strings
    """

    num_unique = len(text1 | text2)
    num_equal = len(text1 & text2)

    return float(num_equal) / float(num_unique)

In [90]:
df = pl.read_csv('raw_data.csv').sort('ID')
df = df.filter(pl.col('text').str.len_chars() > 11)
a = []
for i in df.iter_rows():
    a.append(find_ngrams(i[1]))
df_h2 = pl.DataFrame({'ngram': a})
df = pl.concat([df, df_h2], how='horizontal')

df

ID,text,ngram
i64,str,object
1,"""create a new account log in al…","{'лаг', 'аня', ' m', ' lo', ' jm', 'пар', 'eom', 'eme', 'ver', 'luб', 'мми', 'али', 'acc', 'og ', 'оль', 'ова', ' п', 'wor', 'unt', 'eri', 'jmp', ' al', 'er1', 'mam', 'ty ', 'uб ', 'ико', 'про', 'lab', ' ha', ' па', 'mbe', ' cr', 'пре', '2ki', 'уже', 'cou', 'ber', 'ль ', 'eat', '12k', '996', 'rem', 'ter', 'mem', ' j', 'аро', ' уж', ' у', ' пр', ' ac', 'ity', 'аю ', 'er ', ' i', 'oun', 'rea', 'han', 'огр', ' кр', 'зан', ' ne', 'kig', ' к', 'ли ', 'igf', 'дла', 'уто', 'swo', 'ew ', 'me ', 'mpr', ' a ', ' 1', 'ага', ' n', 'pai', '96 ', 'мир', 'амм', 'bas', ' ни', 'ком', 'ом ', 'ma ', 'rit', 'ank', ' 12', 'klu', ' in', ' c', 'ama', '121', 'rd ', 'рам', ' me', ' p', 'же ', ' ve', ' l', 'гаю', 'нап', ' з', 'иро', 'nt ', '21 ', 'oma', 'pr1', 'рог', '23 ', 'кру', ' на', 'ala', 'in ', 'new', 'ais', ' pa', ' v', 'aba', 'veo', 'ste', 'ров', 'te ', 'tg ', ' за', ' h', 'mp1', 'ord', '123', 'вал', 'log', 'ред', ' re', 'p12', 'рут', 'cre', '199', 'ник', ' н', 'gft', 'r12', 'cco', 'гра', 'ftg', 'ast', 'nkl', ' r', 'апр', 'isw', 'r19', 'ят ', 'emb', 'ate', 'рол', 'едл', 'то ', 'нят', ' a'}"
2,"""they have hostages and they de…","{'hey', 've ', 'hos', 'tag', 'ost', 'ema', '4ok', 'age', ' h', ' ho', ' ca', ' in', ' c', ' th', ' d', 'es ', 'sta', 'cas', ' i', 'man', 'ges', ' de', ' 4', 'the', 'ash', ' an', ' t', ' ha', 'ave', 'ok ', 'dem', 'hav', 'nd ', 'and', 'ey ', 'in ', ' 4o', ' a', 'sh '}"
3,"""co смерmью этпоёо персонажа hu…","{'кля', ' m', 'бы ', 'auo', 'охр', 'ран', ' ш', 'alu', 'зum', ' ua', 'mb ', 'воc', 'ью ', 'хра', 'cя ', 'сmc', ' uc', 'ccm', 'оcc', ' п', 'зал', 'lu ', 'чсн', 'cum', ' вa', ' в', 'про', 'mnь', 'ую ', 'mеч', 'ьбы', 'лят', 'ль ', 'oeu', 'awe', ' чm', 'оpы', ' mе', ' ко', 'рсо', 'окл', 'eum', 'сох', 'ятп', 'нен', 'сон', ' ц ', ' о', 'азр', 'ucu', 'ыu ', 'асm', 'dьб', 'ото', 'uлu', 'uoe', ' пр', 'hum', 'кот', 'eu ', 'узu', 'тпо', 'pe ', 'наж', 'ане', ' u', 'apy', 'алu', 'оёо', ' к', 'обр', 'mau', 'удь', 'тоp', 'wup', 'енн', 'me ', 'nь ', 'cma', ' я ', 'нну', 'нuе', ' я', 'этп', 'суд', 'обы', 'мер', 'руз', 'сме', ' см', ' су', 'cal', 'аль', 'ом ', 'ажа', 'лu ', 'mcя', ' ca', ' c', ' пе', 'вас', 'ечс', 'соз', ' w', ' з', 'co ', 'ерm', 'зда', ' об', ' шe', 'ерс', ' uл', ' э', 'mоб', 'пом', 'дьб', ' ц', 'ume', 'чmо', 'дал', 'ыва', ' co', 'она', 'umb', 'снu', 'уdь', 'пер', 'поё', ' во', 'uap', 'суd', 'озд', 'заз', 'py ', 'ёо ', 'бры', ' за', ' h', 'upe', ' с', ' wu', 'рок', 'рыв', 'uе ', ' ч', 'рmь', 'umn', ' эт', 'mью', 'вaw', 'weu', ' со', 'шe ', 'зру', 'pыu', ' hu', 'жа ', 'ную'}"
4,"""непреодомимое ж еан ие написат…","{' не', 'имо', ' и', ' чг', 'иса', ' еа', 'мим', 'мое', ' ж ', ' 0 ', 'ть ', ' с', 'ать', 'епр', 'гиэ', ' ж', 'дом', ' ие', 'иэ ', 'еан', ' ч', ' в ', 'чги', 'нап', ' с ', 'апи', ' 0', ' н', 'ан ', 'сат', 'пис', ' е', ' в', 'ое ', 'одо', 'ие ', 'еод', 'пре', 'оми', ' на', 'неп', 'рео'}"
5,"""иногда я забываю чо вынепронто…","{'вне', ' ес', 'ня ', 'сча', 'ди ', 'люд', 'ояц', 'кфо', 'елш', 'онт', 'да ', 'нел', 'ппы', 'иен', ' е', ' в', 'про', 'юди', ' кф', ' ци', 'цие', 'ыне', 'ясч', 'елю', 'иеи', 'мы ', 'ишм', 'аст', 'огд', 'мен', 'емэ', 'еив', 'лше', 'ть ', 'нто', 'епр', 'ивн', 'еан', 'аю ', 'фин', 'ане', 'нки', 'мэю', 'рон', ' к', 'ив ', 'ыум', 'мя ', 'тив', 'ваю', 'мви', 'амо', ' я ', 'емн', ' я', 'еня', ' ия', 'заб', 'ине', 'мое', 'выу', 'гда', 'шем', ' ин', 'ино', 'эюп', 'ыиш', ' вы', 'оыи', 'оем', ' з', 'пы ', 'цип', 'ана', 'над', 'мнм', 'быв', 'уме', 'кие', ' ц', ' на', 'вын', 'ыва', 'ест', ' м', 'сти', 'дпи', ' и', 'имя', 'ног', 'час', ' чо', 'ано', 'шмв', 'фоы', 'адп', 'ноя', ' за', 'вям', 'ями', 'юпа', 'сть', 'неа', 'яци', 'ивя', ' ч', 'ми ', ' мф', ' н', 'нмы', 'иши', 'ияс', 'ипп', 'пив', 'нан', 'абы', 'чо ', 'мфи', 'инк', 'ена', 'шин', 'то ', 'нам', ' иш', 'неп', 'вим', 'пан'}"
…,…,…
7336678,"""как я геея пщещам кпгдд енимнн…","{'гее', 'пещ', ' я ', ' я', 'как', 'ени', ' пп', ' гп', 'рпг', 'имн', 'гчл', 'ним', ' г ', 'хра', 'епе', 

In [100]:
sdf = pl.read_csv('raw_data.csv').sort('ID')
sdf = sdf.filter(pl.col('text').str.len_chars() > 11)
sdf.write_csv('new_raw_data.csv')

In [96]:

lst = []

for i, r1 in enumerate(df.iter_rows()):
    for j, r2 in enumerate(df.iter_rows()):

        if j <= i:
            continue
        sim = only_similarity(r1[2], r2[2])
        if 0.8 > sim >= 0.7:
            lst.append([r1[0], r2[0], sim])
lst


AttributeError: 'generator' object has no attribute 'lazy'

In [55]:
df = pl.read_csv('result_hard.csv')
df = df.sort('sim')

error_ocr_memes = df.filter(pl.col('sim') == 1)

In [56]:
dupl_not_proc = df.filter(pl.col('sim').is_between(0.9, 1)).filter(pl.col('sim')!=0.9).filter(pl.col('sim') != 1)

In [70]:
new_dupl = df.filter(pl.col("sim") < 0.81)

final = ''
for row in new_dupl.iter_rows():
    final = f'{final} {row[0]} {row[1]}'
final

' 5012162 475990 3029492 1531348 1407693 6594 4062203 112348 7085824 1299509 1820525 1471444 7062879 1084698 4281807 1194950 3060807 1588884 1407693 5348 2958781 2958582 5386922 4860859 3556472 2554578 5981981 5904404 3358522 2490612 6243424 3763298 3485827 3432906 4175863 4075357 6275189 6253971 5130672 5021571 5314706 5021571 6339449 6195001 5859368 5669818 4902261 4820749 5908683 5272287 6022630 6018119 6531770 6401611 5999113 5651946 6695155 6004068 6324678 6307155 6253971 6158875 6717141 6552965 5611421 5579371 1734025 151 1203129 44878 4767655 129622 786980 276957 2652541 204484 6279055 2554848 1017546 24858 1064403 986983 5729240 5182699 5990542 5772508 543030 300874 6111754 5886038 727053 492955 1185648 1173747 5561902 1179490 4252305 3949600 5895514 5457592 3001358 276957 6730974 5085400 6409614 3346259 313193 5439 4767318 4688021 6925 5563 2490529 563058 2214327 1972567 4322077 2963069 6730963 6076066 3502571 1084434 1084434 1064315 2367200 996255 33771 13722 3350455 195 3858

In [72]:
new_dupl = df.filter(pl.col('sim') < 0.9)

final = ''
for row in new_dupl.iter_rows():
    final = f'{final} {row[0]} {row[1]}'
final

' 5012162 475990 3029492 1531348 1407693 6594 4062203 112348 7085824 1299509 1820525 1471444 7062879 1084698 4281807 1194950 3060807 1588884 1407693 5348 2958781 2958582 5386922 4860859 3556472 2554578 5981981 5904404 3358522 2490612 6243424 3763298 3485827 3432906 4175863 4075357 6275189 6253971 5130672 5021571 5314706 5021571 6339449 6195001 5859368 5669818 4902261 4820749 5908683 5272287 6022630 6018119 6531770 6401611 5999113 5651946 6695155 6004068 6324678 6307155 6253971 6158875 6717141 6552965 5611421 5579371 1734025 151 1203129 44878 4767655 129622 786980 276957 2652541 204484 6279055 2554848 1017546 24858 1064403 986983 5729240 5182699 5990542 5772508 543030 300874 6111754 5886038 727053 492955 1185648 1173747 5561902 1179490 4252305 3949600 5895514 5457592 3001358 276957 6730974 5085400 6409614 3346259 313193 5439 4767318 4688021 6925 5563 2490529 563058 2214327 1972567 4322077 2963069 6730963 6076066 3502571 1084434 1084434 1064315 2367200 996255 33771 13722 3350455 195 3858

In [74]:
new_dupl.write_csv('new_duplicates.csv')
error_ocr_memes.write_csv('error_ocr_memes.csv')